# Monte Carlo Tree Search (MCTS)
This notebook similates an agent that plays with UCT-MCTS and is based on an implementation written by [Matan Tsipory](https://www.kaggle.com/code/matant/monte-carlo-tree-search-connectx?scriptVersionId=37955585). For an overview of Monte Carlo Tree Search, see the [Wikipedia page](https://en.wikipedia.org/wiki/Monte_Carlo_tree_search). Also review the [Python implementation for Tic-Tac-Toe](https://github.com/int8/monte-carlo-tree-search).

A guide for Monte Carlo Tree Search is the [int8.io machine learning blog](https://int8.io/monte-carlo-tree-search-beginners-guide/#Policy_network_training_in_Alpha_Go_and_Alpha_Zero).

# Imports

In [ ]:
# Define ConnectX environment (was defined in v0.1.6)
!pip install 'kaggle-environments>=0.1.6'
from kaggle_environments import evaluate, make, utils
from functools import lru_cache
import numpy as np
import random
import math
import time

# Create ConnectX Environment

In [ ]:
env = make("connectx", debug=True)
configuration = env.configuration
print(configuration)

# Functions for MCTS Agent
The implementation below is based on the methods described in [this paper](https://ieeexplore.ieee.org/document/6145622).

In [ ]:
def MCTS_agent(observation, configuration):
    """
    Connect X agent based on Monte Carlo Tree Search (MCTS), with optimizations
    for immediate win/loss detection and cached win-checking.
    
    Parameters
    ----------
    observation : object
        The current game observation which includes the board state and player mark.
    configuration : object
        Game configuration that contains parameters like the number of rows, columns, 
        in-a-row requirement, and the time limit.
    
    Returns
    -------
    int
        The column number where the agent decides to place the piece.
    """
    import random
    import math
    import time
    global current_state  # so tree can be recycled

    init_time = time.time()
    EMPTY = 0
    T_max = configuration.timeout - 0.34  # Time per move, with overhead for safety
    Cp_base = 1  # Base exploration parameter

    def optimized_play():
        """
        Makes an optimized play decision using immediate win/loss detection,
        row tracking, and batched checks for opponent's winning moves.

        Returns
        -------
        int
            The chosen column index for the best move.
        """
        for col in range(configuration.columns):
            if row_tracker.lowest_row[col] >= 0:
                outcome = check_immediate_outcome(board, col, mark, configuration)
                if outcome == 'win':
                    return col  # Play winning move
                elif outcome == 'loss':
                    continue  # Avoid losing move

        safe_moves = [col for col in range(configuration.columns) if col not in batch_immediate_loss_check(board, mark, configuration)]
        return safe_moves[0] if safe_moves else 0

    def play(board, column, mark, config):
        """
        Executes a move on the board by placing the player's mark in the specified column.

        Parameters
        ----------
        board : list of int
            The current board state represented as a 1D list.
        column : int
            The column in which to place the mark.
        mark : int
            The player's mark (1 or 2).
        config : object
            The game configuration with dimensions.
        """
        columns = config.columns
        rows = config.rows
        row = max([r for r in range(rows) if board[column + (r * columns)] == EMPTY])
        board[column + (row * columns)] = mark

    def is_win(board, column, mark, config):
        """
        Checks if the last move resulted in a win.

        Parameters
        ----------
        board : list of int
            The current board state.
        column : int
            The column where the last mark was placed.
        mark : int
            The player's mark (1 or 2).
        config : object
            The game configuration including win conditions.

        Returns
        -------
        bool
            True if the move results in a win, False otherwise.
        """
        columns = config.columns
        rows = config.rows
        inarow = config.inarow - 1
        row = min([r for r in range(rows) if board[column + (r * columns)] == mark])

        def count(offset_row, offset_col):
            run = 0
            for i in range(1, inarow + 1):
                r, c = row + offset_row * i, column + offset_col * i
                if r < 0 or r >= rows or c < 0 or c >= columns or board[c + r * columns] != mark:
                    break
                run += 1
            return run
    
        return (
                count(1, 0) >= inarow  # vertical.
                or (count(0, 1) + count(0, -1)) >= inarow  # horizontal.
                or (count(-1, -1) + count(1, 1)) >= inarow  # top left diagonal.
                or (count(-1, 1) + count(1, -1)) >= inarow  # top right diagonal.
        )

    @lru_cache(maxsize=10000)
    def is_win_cached(board, column, mark, config):
        """
        Checks if placing a piece in the specified column leads to a win for the specified player,
        using a cached version of the is_win function to improve efficiency.
    
        Parameters
        ----------
        board : np.ndarray
            The current board state.
        column : int
            Column index where the piece was placed.
        mark : int
            The player's mark (1 or 2).
        config : Config
            Game configuration, including the win length and board dimensions.
    
        Returns
        -------
        bool
            True if placing the piece results in a win for the player, otherwise False.
        """
        return is_win(board, column, mark, config)

    def play_and_check_win(board, column, mark, config):
        """
        Plays a piece in the specified column and checks if it results in a win.
    
        Parameters
        ----------
        board : np.ndarray
            The current board state.
        column : int
            Column index where the piece is to be placed.
        mark : int
            The player's mark (1 or 2).
        config : Config
            Game configuration, including the win length and board dimensions.
    
        Returns
        -------
        bool
            True if placing the piece results in a win for the player, otherwise False.
        """
        temp_board = board.copy()
        play(temp_board, column, mark, config)
        return is_win_cached(temp_board, column, mark, config)

    def check_immediate_outcome(board, column, mark, config):
        """
        Determines if playing in the specified column leads to an immediate win
        for the current player or loss (win for opponent).
    
        Parameters
        ----------
        board : np.ndarray
            The current board state.
        column : int
            Column index where the piece is to be placed.
        mark : int
            The player's mark (1 or 2).
        config : Config
            Game configuration, including the win length and board dimensions.
    
        Returns
        -------
        str or None
            'win' if the move results in a win for the player, 'loss' if the move leads to an
            immediate win for the opponent, and None if neither is true.
        """
        if play_and_check_win(board, column, mark, config):
            return 'win'
        
        opponent = opponent_mark(mark)
        if play_and_check_win(board, column, opponent, config):
            return 'loss'
        
        return None

    def batch_immediate_loss_check(board, mark, config):
        """
        Checks all columns to see if any of them leads to an immediate loss (win for opponent)
        in one function call, reducing redundant play-check cycles.
    
        Parameters
        ----------
        board : np.ndarray
            The current board state.
        mark : int
            The player's mark (1 or 2).
        config : Config
            Game configuration, including the win length and board dimensions.
    
        Returns
        -------
        list of int
            List of columns that would result in an immediate loss if played.
        """
        opponent = opponent_mark(mark)
        losing_moves = []
        for column in range(config.columns):
            if board[0][column] == 0:  # Only consider non-full columns
                temp_board = board.copy()
                play(temp_board, column, opponent, config)
                if is_win_cached(temp_board, column, opponent, config):
                    losing_moves.append(column)
        return losing_moves

    def is_tie(board):
        """
        Checks if the board is full, resulting in a tie.

        Parameters
        ----------
        board : list of int
            The current board state.

        Returns
        -------
        bool
            True if the board is full, indicating a tie.
        """
        return not(any(mark == EMPTY for mark in board))

    def check_finish_and_score(board, column, mark, config):
        """
        Evaluates if the game has finished and provides a score.

        Parameters
        ----------
        board : list of int
            The current board state.
        column : int
            The column of the last move.
        mark : int
            The player's mark.
        config : object
            The game configuration.

        Returns
        -------
        tuple
            (bool, float): First element is True if the game is finished; the second is the score.
        """
        if is_win(board, column, mark, config):
            return (True, 1)
        if is_tie(board):
            return (True, 0.5)
        else:
            return (False, None)

    def uct_score(node_total_score, node_total_visits, parent_total_visits, Cp=Cp_base):
        """
        Calculates the UCB1 score for balancing exploration and exploitation.

        Parameters
        ----------
        node_total_score : float
            Total score accumulated by the node.
        node_total_visits : int
            Number of visits to the node.
        parent_total_visits : int
            Number of visits to the parent node.
        Cp : float, optional
            Exploration constant, by default Cp_base.

        Returns
        -------
        float
            The UCB1 score.
        """
        if node_total_visits == 0:
            return math.inf
        return node_total_score / node_total_visits + Cp * math.sqrt(
            2 * math.log(parent_total_visits) / node_total_visits)

    def opponent_mark(mark):
        """
        Determines the opponent's mark.
    
        Parameters
        ----------
        mark : int
            The mark of the current player (1 or 2).
    
        Returns
        -------
        int
            The mark of the opponent player (2 if the input mark is 1, and 1 if the input mark is 2).
        """
        return 3 - mark

    def opponent_score(score):
        """
        Returns the opponent's score, used in score backpropagation in the MCTS process.
    
        Parameters
        ----------
        score : float
            The score of the current player in the MCTS simulation (ranging from 0.0 to 1.0).
    
        Returns
        -------
        float
            The score from the opponent's perspective, calculated as `1 - score`.
        """
        return 1 - score

    def random_action(board, config):
        """
        Selects a random legal move from the available columns, avoiding moves that would 
        immediately lose the game.
    
        Parameters
        ----------
        board : list of int
            The current board state represented as a 1D list.
        config : object
            The game configuration containing parameters such as the number of columns and rows.
    
        Returns
        -------
        int
            The column index of a randomly selected legal move.
        """
        legal_moves = [c for c in range(config.columns) if board[c] == EMPTY]
        safe_moves = [c for c in legal_moves if not is_immediate_loss(board, c, config)]
        return random.choice(safe_moves) if safe_moves else random.choice(legal_moves)

    def is_immediate_loss(board, column, config):
        """
        Check if placing a piece in the specified column results in an immediate loss for the player.
    
        This function temporarily plays a piece for the opponent in the given column and checks if it
        creates a winning condition for the opponent. It is useful to filter out moves that would 
        allow the opponent to win immediately in their next turn.
    
        Parameters
        ----------
        board : list of int
            The current state of the game board represented as a 1D list.
            Each element corresponds to a slot, where an integer indicates the player mark 
            or an empty slot.
        column : int
            The column index where the piece is considered to be placed.
        config : object
            The game configuration object containing board dimensions and win conditions. 
            Attributes include:
                - `columns` (int): The number of columns on the board.
                - `rows` (int): The number of rows on the board.
                - `inarow` (int): The number of consecutive pieces required to win.

        Returns
        -------
        bool
            True if placing a piece in the specified column results in an immediate loss,
            meaning the opponent can win in the next turn; False otherwise.
    
        Notes
        -----
        This function creates a temporary copy of the board and does not modify the original board state.
        It assumes that `play` and `is_win` functions are available to simulate moves and check for 
        winning conditions, respectively.
        """
        temp_board = board.copy()
        play(temp_board, column, opponent_mark(observation.mark), config)
        return is_win(temp_board, column, opponent_mark(observation.mark), config)

    def default_policy_simulation(board, mark, config):
        """
        Simulates a game using random moves until completion, returning the score.

        Parameters
        ----------
        board : list of int
            Current board state.
        mark : int
            Starting player's mark.
        config : object
            Game configuration.

        Returns
        -------
        float
            Resulting game score for the starting player.
        """
        original_mark = mark
        board = board.copy()
        column = random_action(board, config)
        play(board, column, mark, config)
        is_finish, score = check_finish_and_score(board, column, mark, config)
        while not is_finish:
            mark = opponent_mark(mark)
            column = random_action(board, config)
            play(board, column, mark, config)
            is_finish, score = check_finish_and_score(board, column, mark, config)
        if mark == original_mark:
            return score
        return opponent_score(score)
    
    def find_action_taken_by_opponent(new_board, old_board, config):
        """
        Identifies the column where the opponent has made a move, based on the difference
        between the new and old board states. This function is used for reusing the MCTS tree 
        between turns.
    
        Parameters
        ----------
        new_board : list of int
            The updated board state after the opponent's move.
        old_board : list of int
            The board state before the opponent's move.
        config : object
            Game configuration, which includes properties like the number of columns.
    
        Returns
        -------
        int
            The column index where the opponent placed their piece.
            Returns -1 if no difference is found, which should not occur in normal gameplay.
        """
        for i, piece in enumerate(new_board):
            if piece != old_board[i]:
                return i % config.columns
        return -1  # shouldn't get here

    def dynamic_cp_adjustment(children_scores, base_cp=Cp_base, max_increase=3, min_cp=0.1):
        """
        Adjusts Cp dynamically based on the variance in children scores.
    
        Parameters
        ----------
        children_scores : list of float
            List of scores from child nodes of the current state.
        base_cp : float, optional
            Base Cp for exploration, by default Cp_base.
        max_increase : float, optional
            Maximum factor by which to increase Cp, by default 3.
        min_cp : float, optional
            Minimum Cp value to maintain exploration, by default 0.1.
    
        Returns
        -------
        float
            Adjusted Cp based on variance in child scores, capped at max_increase and min_cp.
        """
        if len(children_scores) < 2:  # Avoid calculating variance without at least 2 scores
            return base_cp
    
        # Calculate the mean of the children_scores
        mean_score = sum(children_scores) / len(children_scores)
        
        # Calculate the standard deviation
        variance = sum((x - mean_score) ** 2 for x in children_scores) / len(children_scores)
        std_dev = variance ** 0.5

        # Adjust Cp based on the standard deviation
        adjusted_cp = base_cp * (1 + std_dev)
        adjusted_cp = max(adjusted_cp, min_cp)  # Upper bound of cp
        adjusted_cp = min(adjusted_cp, base_cp * max_increase)  # Lower bound of cp
    
        return adjusted_cp

    class State():
        """
        Represents a node in the game tree for Monte Carlo Tree Search (MCTS).

        Attributes
        ----------
        board : list of int
            The current board state represented as a 1D list.
        mark : int
            The current player's mark (1 or 2).
        config : object
            Game configuration, containing parameters like rows, columns, and in-a-row requirement.
        parent : State, optional
            The parent node in the tree. None if this is the root.
        children : list of State
            List of child nodes representing possible moves from the current state.
        node_total_score : float
            The cumulative score of this node from all simulations.
        node_total_visits : int
            The number of times this node has been visited during MCTS.
        is_terminal : bool
            Indicates whether this node represents a terminal state (win or tie).
        terminal_score : float, optional
            Score of the terminal state if `is_terminal` is True.
        action_taken : int, optional
            The column index of the move taken to reach this state from the parent.
        available_moves : list of int
            List of columns where a move can be legally played from this state.
        expandable_moves : list of int
            Subset of `available_moves` that have not yet been explored.
        """
        def __init__(self, board, mark, config, parent=None, is_terminal=False, terminal_score=None, action_taken=None):
            """
            Initializes a State instance.
    
            Parameters
            ----------
            board : list of int
                The current board state.
            mark : int
                The player's mark (1 or 2).
            config : object
                Game configuration, containing parameters like rows, columns, and in-a-row requirement.
            parent : State, optional
                The parent state in the MCTS tree, by default None.
            is_terminal : bool, optional
                Whether this state is a terminal node, by default False.
            terminal_score : float, optional
                The score of the terminal state, if applicable, by default None.
            action_taken : int, optional
                The column index of the action taken to reach this state, by default None.
            """
            self.board = board.copy()
            self.mark = mark
            self.config = config
            self.children = []
            self.parent = parent
            self.node_total_score = 0
            self.node_total_visits = 0
            self.available_moves = [c for c in range(config.columns) if board[c] == EMPTY]
            self.expandable_moves = self.available_moves.copy()
            self.is_terminal = is_terminal
            self.terminal_score = terminal_score
            self.action_taken = action_taken

        def is_expandable(self):
            """
            Checks if the current node has unexplored child nodes, which indicates
            the node can still be expanded by exploring unvisited moves.
    
            Returns
            -------
            bool
                True if there are still moves to explore from this state; False otherwise.
            """
            return (not self.is_terminal) and (len(self.expandable_moves) > 0)

        def expand_and_simulate_child(self):
            """
            Expands a new child node by selecting a random move from `expandable_moves`,
            then simulates the game outcome from this child node and backpropagates the score.
    
            This method performs the Expansion, Simulation, and Backpropagation steps of MCTS.
            """
            column = random.choice(self.expandable_moves)
            child_board = self.board.copy()
            play(child_board, column, self.mark, self.config)
            is_terminal, terminal_score = check_finish_and_score(child_board, column, self.mark, self.config)
            self.children.append(State(child_board, opponent_mark(self.mark),
                                       self.config, parent=self,
                                       is_terminal=is_terminal,
                                       terminal_score=terminal_score,
                                       action_taken=column
                                       ))
            simulation_score = self.children[-1].simulate()
            self.children[-1].backpropagate(simulation_score)
            self.expandable_moves.remove(column)

        def choose_strongest_child(self, base_cp):
            """
            Selects the child node with the highest Upper Confidence Bound (UCB1) score 
            using dynamically adjusted Cp.
        
            Parameters
            ----------
            base_cp : float
                The base exploration parameter for UCB1.
        
            Returns
            -------
            State
                The child node with the highest UCB1 score.
            """
            # Calculate initial UCB1 scores for each child with the given Cp
            children_scores = [uct_score(child.node_total_score,
                                          child.node_total_visits,
                                          self.node_total_visits,
                                          base_cp) for child in self.children]
            
            # Dynamically adjust Cp based on the calculated scores
            dynamic_cp = dynamic_cp_adjustment(children_scores, base_cp)
            
            # Recalculate UCB1 scores with the dynamically adjusted Cp
            adjusted_scores = [uct_score(child.node_total_score,
                                          child.node_total_visits,
                                          self.node_total_visits,
                                          dynamic_cp) for child in self.children]
            
            # Find the index of the child with the maximum adjusted score
            best_child_index = adjusted_scores.index(max(adjusted_scores))
            
            return self.children[best_child_index]
            
        def choose_play_child(self):
            """
            Selects the child node with the highest cumulative score as the 
            next move choice for the agent.
    
            Returns
            -------
            State
                The child node with the highest cumulative score.
            """
            children_scores = [child.node_total_score for child in self.children]
            max_score = max(children_scores)
            best_child_index = children_scores.index(max_score)
            return self.children[best_child_index]

        def tree_single_run(self):
            """
            Executes a single iteration of the MCTS process, encompassing the 
            selection, expansion, simulation, and backpropagation phases.
    
            If the current node is terminal, it only backpropagates its score.
            Otherwise, it either expands a child node if possible or continues 
            to traverse the tree by selecting the strongest child.
    
            Returns
            -------
            None
            """
            if self.is_terminal:
                self.backpropagate(self.terminal_score)
                return
            if self.is_expandable():
                self.expand_and_simulate_child()
                return
            self.choose_strongest_child(Cp_base).tree_single_run()

        def simulate(self):
            """
            Runs a simulation (playout) from the current node to a terminal state using random moves.
            
            If the node is terminal, returns its score; otherwise, calls the default policy simulation.
    
            Returns
            -------
            float
                The resulting score from the simulation for the current player.
            """
            if self.is_terminal:
                return self.terminal_score
            return opponent_score(default_policy_simulation(self.board, self.mark, self.config))

        def backpropagate(self, simulation_score):
            """
            Updates this node's score and visit count, then propagates the updated score to its ancestors.
    
            Parameters
            ----------
            score : float
                The score resulting from a simulation to be backpropagated.
            """
            self.node_total_score += simulation_score
            self.node_total_visits += 1
            if self.parent is not None:
                self.parent.backpropagate(opponent_score(simulation_score))
                
        def choose_child_via_action(self, action):
            """
            Retrieves the child node that corresponds to a specific action taken 
            from the current state, allowing tree reuse between agent moves.
    
            Parameters
            ----------
            action : int
                The column index of the move that was taken to transition to this child.
    
            Returns
            -------
            State or None
                The child node representing the given action, or None if no child matches.
            """
            for child in self.children:
                if child.action_taken == action:
                    return child
            return None

    class RowTracker:
        """
        Tracks the lowest available row in each column of the board, allowing quick placement
        of pieces without iterating over each row.
    
        Attributes
        ----------
        lowest_row : list of int
            Tracks the lowest empty row for each column.
        config : Config
            Game configuration, including the win length and board dimensions.
        """
    
        def __init__(self, board, config):
            """
            Initializes RowTracker with the current board state and configuration.
    
            Parameters
            ----------
            board : np.ndarray
                The current board state.
            config : Config
                Game configuration, including the win length and board dimensions.
            """
            self.config = config
            self.lowest_row = [self.find_lowest_row(board, col) for col in range(config.columns)]
    
        def find_lowest_row(self, board, column):
            """
            Finds the lowest empty row in a specified column.
    
            Parameters
            ----------
            board : np.ndarray
                The current board state.
            column : int
                The column index.
    
            Returns
            -------
            int
                The lowest empty row index, or -1 if the column is full.
            """
            for row in range(self.config.rows - 1, -1, -1):
                if board[row][column] == 0:
                    return row
            return -1
    
        def update_after_play(self, column):
            """
            Updates the lowest row tracker after a piece is played in the specified column.
    
            Parameters
            ----------
            column : int
                Column index where the piece was placed.
            """
            self.lowest_row[column] -= 1

    board = observation.board
    mark = observation.mark
    
    # If current_state already exists, recycle it based on action taken by opponent
    try:  
        current_state = current_state.choose_child_via_action(
            find_action_taken_by_opponent(board, current_state.board, configuration))
        current_state.parent = None  # make current_state the root node, dereference parents and siblings
        
    except:  # new game or other error in recycling attempt due to Kaggle mechanism
        current_state = State(board, mark,  # This state is considered after the opponent's move
                              configuration, parent=None, is_terminal=False, terminal_score=None, action_taken=None)
   
    # Run MCTS iterations until time limit is reached.
    while time.time() - init_time <= T_max:
        current_state.tree_single_run()
        
    current_state = current_state.choose_play_child()
    return current_state.action_taken

# Check agent validity

In [ ]:
def reset_env():
    try:
        del current_state  # Only delete if current_state has been assigned
    except NameError:
        pass  # Ignore if current_state is not defined

# Reset the environment and clear any existing tree state
reset_env()

# Run the environment
env.run([MCTS_agent, MCTS_agent])

# Check if the agents have completed successfully
print("Success!" if env.state[0].status == env.state[1].status == "DONE" else "Failed...")

# Evaluate your agent

In [ ]:
def mean_reward(rewards):
    """
    Calculates the mean reward for a set of rewards, weighted by the number of trials.

    Parameters
    ----------
    rewards : list of tuple of int
        A list of tuples where each tuple contains two integers:
        - The first integer represents the total number of successful trials (rewards).
        - The second integer represents the total number of unsuccessful trials.

    Returns
    -------
    float
        The mean reward, calculated as the sum of successful trials divided by
        the sum of all trials (both successful and unsuccessful).
    
    Notes
    -----
    This function computes a weighted mean, where each reward tuple's weight is
    determined by the total number of trials in that tuple.
    
    Example
    -------
    >>> mean_reward([(3, 2), (4, 1), (2, 3)])
    0.6
    """
    # Calculate the total number of successful trials across all rewards
    total_rewards = sum(r[0] for r in rewards)
    
    # Calculate the total number of trials (successful + unsuccessful)
    total_trials = sum(r[0] + r[1] for r in rewards)
    
    # Return the weighted mean of successful trials
    return total_rewards / total_trials
    

# Run multiple episodes to estimate its performance.
print("My Agent vs Random Agent:", mean_reward(evaluate("connectx", [MCTS_agent, "random"], num_episodes=20)))
print("Random Agent vs My Agent:", mean_reward(evaluate("connectx", ["random", MCTS_agent], num_episodes=20)))
print("My Agent vs Negamax Agent:", mean_reward(evaluate("connectx", [MCTS_agent, "negamax"], num_episodes=5)))
print("Negamax Agent vs My Agent:", mean_reward(evaluate("connectx", ["negamax", MCTS_agent], num_episodes=5)))

> # Play your Agent
Click on any column to place a checker there ("manually select action").

In [ ]:
# "None" represents which agent you'll manually play as (first or second player).
env.play([MCTS_agent, None], width=500, height=450)

# Write Submission File



In [ ]:
import inspect

submission_path = "submission.py"
        
def write_agent_to_file(function, file):
    with open(file, "w") as f:
        f.write("from functools import lru_cache\n")
        f.write("import numpy as np\n")
        f.write(inspect.getsource(function))
        print(function, "written to", file)

write_agent_to_file(MCTS_agent, submission_path)

# Validate Submission
Play your submission against itself.  This is the first episode the competition will run to weed out erroneous agents.

Why validate? This roughly verifies that your submission is fully encapsulated and can be run remotely.

In [ ]:
# Note: Stdout replacement is a temporary workaround.
import sys
out = sys.stdout
try:
    submission = utils.read_file("/kaggle/working/submission.py")
    agent = utils.get_last_callable(submission)
finally:
    sys.stdout = out

env = make("connectx", debug=True)
env.run([agent, agent])
print("Success!" if env.state[0].status == env.state[1].status == "DONE" else "Failed...")